In [1]:
import tensorflow as tf
from tensorflow import keras

keras.__version__

'2.12.0'

In [2]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[28, 28]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    model.compile(loss=keras.losses.MSE, optimizer=keras.optimizers.legacy.SGD(learning_rate=0.01))
    return model



In [3]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)
keras_reg

<ipython-input-3-9dabefbb1669>:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


In [4]:
reg = build_model()
reg

In [5]:
reg

In [6]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [7]:
keras_reg.fit(X_train, y_train, epochs=5,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])
mse_test = keras_reg.score(X_test, y_test)


Epoch 1/5
1719/1719 [==============================] - 14s 4ms/step - loss: 8.4197 - val_loss: 8.2983
Epoch 2/5
1719/1719 [==============================] - 7s 4ms/step - loss: 8.2912 - val_loss: 8.2344
Epoch 3/5
1719/1719 [==============================] - 7s 4ms/step - loss: 8.2861 - val_loss: 8.2580
Epoch 4/5
1719/1719 [==============================] - 7s 4ms/step - loss: 8.2798 - val_loss: 8.2841
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 13610.5596


In [9]:
X_new = X_test[:3]
y_pred = keras_reg.predict(X_new)
y_pred

1/1 [==============================] - 0s 37ms/step


array([[  4.660604 ,   4.660604 ,   4.660604 ,   4.660604 ,   4.660604 ,
          4.660604 ,   4.660604 ,   6.6665926,  21.043999 ,  51.475758 ,
         54.224754 ,  49.848873 ,  52.150608 ,  53.793674 ,  69.78407  ,
         91.92359  , 102.39495  , 104.88954  , 111.31415  , 124.40522  ,
        188.72655  , 134.27751  ,   4.660604 ,   4.660604 ,   4.660604 ,
          4.660604 ,   4.660604 ,   4.660604 ],
       [ 14.459228 , 141.14473  , 181.50896  , 183.22072  , 190.20981  ,
        237.55664  , 201.1624   , 202.1519   , 215.60431  , 255.04643  ,
        170.72606  , 181.7914   , 209.63693  , 193.66449  , 180.24315  ,
        179.64044  , 185.05385  , 189.95988  , 193.98984  , 199.57202  ,
        202.779    , 210.79977  , 211.21957  , 201.24683  , 201.0889   ,
         82.292404 ,  86.05458  ,  66.43871  ],
       [ 84.17106  , 144.40959  , 135.5327   , 130.08931  , 126.578995 ,
        129.11444  , 142.8887   , 150.8279   , 136.53885  , 140.80379  ,
        140.99544  , 146.533

In [11]:
import numpy as np
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=3,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/3
1146/1146 [==============================] - 6s 4ms/step - loss: 8.5412 - val_loss: 8.2501
Epoch 2/3
1146/1146 [==============================] - 5s 4ms/step - loss: 8.3287 - val_loss: 8.3376
Epoch 3/3
573/573 [==============================] - 2s 3ms/step - loss: 8.1973
Epoch 1/3
1146/1146 [==============================] - 5s 4ms/step - loss: 8.4728 - val_loss: 8.2353
Epoch 2/3
1146/1146 [==============================] - 5s 4ms/step - loss: 8.2796 - val_loss: 8.2364
Epoch 3/3
573/573 [==============================] - 2s 3ms/step - loss: 8.3105
Epoch 1/3
1146/1146 [==============================] - 5s 4ms/step - loss: 8.4893 - val_loss: 8.2332
Epoch 2/3
1146/1146 [==============================] - 5s 4ms/step - loss: 8.2881 - val_loss: 8.2607
Epoch 3/3
573/573 [==============================] - 2s 3ms/step - loss: 8.3030
Epoch 1/3
1146/1146 [==============================] - 13s 11ms/step - loss: 8.4983 - val_loss: 8.2318
Epoch 2/3
1146/1146 [==============================

KeyboardInterrupt: 